</center><img src="coverage_workflow_image.png"  width="1000" height="300"/></center>

In [2]:
# import required python libraries
#import geopandas as gpd  # for geo-data structure
import numpy as np  # for some operation better done with numpy
import matplotlib.pyplot as plt  # for creating plots
#import shapely.geometry  # for geometry data crunching
#from shapely.geometry import Point  # specifically for point geometry data manipulation
#import keplergl
#from keplergl import KeplerGl
import datacompy
import pandas as pd
crs = {'init': 'epsg:4326'}
#from shapely.geometry import Point
import mpl_toolkits


In [3]:
# bring in the 3 to 4 different coverage dataset from FB, GSMA, ITU and Ericsson as well as the master school_geolocation

In [4]:
d_f = pd.read_csv('./data/Brazil_Connectivity.csv',low_memory=False,encoding='utf-8')
d_i = pd.read_csv('./data/BRA_coverage_ITU.csv',low_memory=False,encoding='utf-8')
#d_e = pd.read_csv('./data/CR_20210301-20210531_Ericsson.csv',low_memory=False,encoding='utf-8')
#d_g = pd.read_csv('./data/GSMA_RW_Schools_mobile_coverage_v1.csv',low_memory=False,encoding='utf-8')
s_master = pd.read_csv('./data/BR_school_geolocation_master.csv',low_memory=False,encoding='utf-8')

In [6]:
d_i.head(2)

,uuid,country,source,giga_school_id,source_school_id,school_name,latitude,longitude,hex8,source_timestamp,...,nearest_UMTS_id,nearest_UMTS_distance,nearest_GSM_id,nearest_GSM_distance,2G,3G,4G,pop_within_1km,pop_within_3km,pop_within_7km
0,82679ff0-f358-33d0-b3ce-4fc4c2740b7e,Brazil,government,G_BRA_000001,11000023,EEEE ABNAEL MACHADO DE LIMA - CENE,-8.7585,-63.8540,888a05915dfffff,2021-04,...,93737,0.009143,48934,0.295171,1,1,1,NaN,NaN,NaN
1,115d27ba-82a1-3744-a3fb-90f216f54f6f,Brazil,government,G_BRA_000002,11000040,EMEIEF PEQUENOS TALENTOS,-8.7937,-63.8839,888a059319fffff,2021-04,...,1768052,0.060195,1672021,0.078968,1,1,1,NaN,NaN,NaN


In [55]:
#d_e.rename(columns={'SCHOOL_CODE':'giga_school_id', 'SCHOOL_NAME':'school_name', '2G_coverage':'2G_quality','3G_coverage':'3G_quality','4G_coverage':'4G_quality'}, inplace=True)
#d_i.rename(columns={'name':'school_name'}, inplace=True)
#d_f.rename(columns={'school_id': 'giga_school_id','name':'school_name'}, inplace=True)

In [56]:
# cross-profiles the sources with master data for completeness and correctness

In [7]:
# create a separate dfs
master = s_master.copy()
fb = d_f.copy()
ec = d_e.copy()
itu = d_i.copy()
#gsma = d_g.copy()
# for facebook
cross_profile_fb_master = datacompy.Compare(
master,
fb,
join_columns='giga_school_id',  
df1_name='Master', 
df2_name='Facebook'
)
#for ericsson
cross_profile_e_master = datacompy.Compare(
master,
ec,
join_columns='giga_school_id',  
df1_name='Master', 
df2_name='Ericsson'
)
# for ITU
cross_profile_itu_master = datacompy.Compare(
master,
itu,
join_columns='giga_school_id',  
df1_name='Master', 
df2_name='ITU'
)
print(cross_profile_fb_master.report())
print(cross_profile_e_master.report())
print(cross_profile_itu_master.report())
#print(cross_profile_gsma_master.report())

NameError: name 'd_e' is not defined

In [58]:
# some colunm names standardization

In [59]:
#d_e.rename(columns={'SCHOOL_CODE':'school_id', 'SCHOOL_NAME':'school_name'}, inplace=True)
#d_i.rename(columns={'giga_schoo': 'giga_school_id','source_sch':'school_id', 'name':'school_name'}, inplace=True)
#d_g.rename(columns={'giga_schoo': 'giga_school_id'}, inplace=True)

In [60]:
# cross-profiles the sources with master data for completeness and correctness

In [61]:
# they usually don't come with "coverage" colunms, we create it at source level using the following codes

In [62]:
#d_e['coverage'] = np.where((d_e['2G_coverage']== 'None') & (d_e['3G_coverage']== 'None')& (d_e['4G_coverage']== 'None'), 'NO', 'YES')
d_f['coverage'] = np.where((d_f['percent_2g'].isnull()) & (d_f['percent_3g'].isnull())& (d_f['percent_4g'].isnull()), 'NO', 'YES')
d_f['coverage'] = np.where((d_f['percent_2g']<=0) & (d_f['percent_3g']<=0) & (d_f['percent_4g']<=0), 'NO', 'YES')
d_i['coverage'] = np.where((d_i['2G'].isnull()) & (d_i['3G'].isnull())& (d_i['4G'].isnull()), 'NO', 'YES')
d_i['coverage'] = np.where((d_i['2G']<=0) & (d_i['3G']<=0) & (d_i['4G']<=0), 'NO', 'YES')


In [63]:
# create the 2G, 3G and 4G coverage columns with binary values at source level using the following codes

In [64]:
d_f['2G_coverage'] = np.where((d_f['percent_2g'] > 0), 'YES', 'NO')
d_f['3G_coverage'] = np.where((d_f['percent_3g'] > 0), 'YES', 'NO')
d_f['4G_coverage'] = np.where((d_f['percent_4g'] > 0), 'YES', 'NO')

In [65]:
d_i['2G_coverage'] = np.where((d_i['2G'] >=1), 'YES', 'NO')
d_i['3G_coverage'] = np.where((d_i['3G'] == 1), 'YES', 'NO')
d_i['4G_coverage'] = np.where((d_i['4G'] ==1), 'YES', 'NO')

In [66]:
#d_e.rename(columns={'2G_coverage':'2G_coverage_quality', '3G_coverage':'3G_coverage_quality', '4G_coverage': '4G_coverage_quality'}, inplace=True)

In [67]:
# they usually don't come with "coverage_type" colunms, we create it at source level using the following function and codes

In [68]:
def coverage_type(df):
    if df['4G_coverage'] == 'YES':
        return "4G"
    elif df['3G_coverage'] == 'YES':
        return "3G"
    elif df['2G_coverage'] == 'YES':
        return "2G"
    else:
        return "None"

In [69]:
d_f['coverage_type'] = d_f.apply((lambda x: coverage_type(x)),axis=1)
#d_e['coverage_type'] = d_e.apply((lambda x: coverage_type(x)),axis=1)
d_i['coverage_type'] = d_i.apply((lambda x: coverage_type(x)),axis=1)


In [70]:
d_e.head(1)

,giga_school_id,school_name,school_region,LATITUDE,LONGITUDE,connectivity,uuid,Coverage,Coverage_type,2G_quality,...,2G_Best_Coverage_provider,3G_Best_Coverage_provider,4G_Best_Coverage_provider,5G_Best_Coverage_provider,2G_coverage_present,3G_coverage_present,4G_coverage_present,5G_coverage_present,Distance_to_nearest_tower,Data_Source
0,G_CRI_000040,C.T.P. DE JACO,Urban,9.627762,-84.623144,YES,973a0ff3-de89-3d61-9cbf-7e1aa3ff480b,YES,4G,Fair,...,Claro,Claro,movistar,None,YES,YES,YES,No,-1,Ericsson


In [71]:
d_e.rename(columns={'2G_coverage_present':'2G_coverage', '3G_coverage_present':'3G_coverage','4G_coverage_present':'4G_coverage', 'Coverage':'coverage', 'Coverage_type':'coverage_type'}, inplace=True)

In [72]:
# isolating the needed colunms from the rest

In [73]:
features = ['giga_school_id','coverage','coverage_type','2G_coverage','3G_coverage','4G_coverage'] 

In [74]:
d_fc = d_f[features]
#d_ec = d_e[features]
d_it = d_i[features]
d_ec = d_e[features]

In [31]:
# more sanity checks for nan on giga_school_id

In [75]:
d_fc = d_fc[d_fc['giga_school_id'].notna()]
#d_ec = d_ec[d_ec['giga_school_id'].notna()]
d_it = d_it[d_it['giga_school_id'].notna()]
d_ec = d_ec[d_ec['giga_school_id'].notna()]

In [76]:
# visually comparing the sources for matches and mismatches across the columns

In [77]:
df_all = pd.concat([d_fc.set_index('giga_school_id'), d_ec.set_index('giga_school_id'),d_it.set_index('giga_school_id')], 
                   axis='columns', keys=['Facebook', 'Ericsson', 'ITU'])
df_all.head()

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


Facebook                                                    \
             coverage coverage_type 2G_coverage 3G_coverage 4G_coverage   
G_CRI_000001      YES            4G         YES         YES         YES   
G_CRI_000002      YES            4G         YES         YES         YES   
G_CRI_000003      YES            4G         YES         YES         YES   
G_CRI_000004      YES            4G         YES         YES         YES   
G_CRI_000005      YES            4G         YES         YES         YES   

             Ericsson                                                    \
             coverage coverage_type 2G_coverage 3G_coverage 4G_coverage   
G_CRI_000001      YES            4G         YES         YES         YES   
G_CRI_000002      YES            4G         YES         YES         YES   
G_CRI_000003      YES            4G         YES         YES         YES   
G_CRI_000004      YES            4G         YES         YES         YES   
G_CRI_000005      YES            4G         YES         YES         YES   

                  ITU                                                    
             coverage coverage_type 2G_coverage 3G_coverage 4G_coverage  
G_CRI_000001      YES            4G         YES         YES         YES  
G_CRI_000002      YES            4G         YES         YES         YES  
G_CRI_000003      YES            4G         YES         YES         YES  
G_CRI_000004      YES            4G         YES         YES         YES  
G_CRI_000005      YES            4G         YES         YES         YES

In [78]:
d_ec.head()

,giga_school_id,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage
0,G_CRI_000040,YES,4G,YES,YES,YES
1,G_CRI_000077,YES,4G,YES,YES,YES
2,G_CRI_000078,YES,4G,YES,YES,YES
3,G_CRI_000100,YES,4G,YES,YES,YES
4,G_CRI_000168,YES,4G,YES,YES,YES


In [79]:
df_final = df_all.swaplevel(axis='columns')[d_fc.columns[1:]]
df_final.head()

coverage               coverage_type              2G_coverage  \
             Facebook Ericsson  ITU      Facebook Ericsson ITU    Facebook   
G_CRI_000001      YES      YES  YES            4G       4G  4G         YES   
G_CRI_000002      YES      YES  YES            4G       4G  4G         YES   
G_CRI_000003      YES      YES  YES            4G       4G  4G         YES   
G_CRI_000004      YES      YES  YES            4G       4G  4G         YES   
G_CRI_000005      YES      YES  YES            4G       4G  4G         YES   

                           3G_coverage               4G_coverage                
             Ericsson  ITU    Facebook Ericsson  ITU    Facebook Ericsson  ITU  
G_CRI_000001      YES  YES         YES      YES  YES         YES      YES  YES  
G_CRI_000002      YES  YES         YES      YES  YES         YES      YES  YES  
G_CRI_000003      YES  YES         YES      YES  YES         YES      YES  YES  
G_CRI_000004      YES  YES         YES      YES  YES         YES      YES  YES  
G_CRI_000005      YES  YES         YES      YES  YES         YES      YES  YES

In [80]:
def highlight_diff(data, color='lightcoral'):
    attr = 'background-color: {}'.format(color)
    other = data.xs('Facebook', axis='columns', level=-1)
    return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                        index=data.index, columns=data.columns)

#df_final.style.apply(highlight_diff, axis=None)

In [81]:
# coverage comparison report

In [83]:
# create a separate dfs
fbc = d_fc.copy()
#ecc = d_ec.copy()
ituc = d_it.copy()
ecc = d_ec.copy()

# Facebook&ITU
cross_profile_fb_itu = datacompy.Compare(
fbc,
ituc,
join_columns='giga_school_id',  
df1_name='Facebook', 
df2_name='ITU'
)

cross_profile_facebook_ec = datacompy.Compare(
fbc,
ecc,
join_columns='giga_school_id',  
df1_name='Facebook', 
df2_name='GSMA'
)


#print(cross_profile_fb_ec.report())
print(cross_profile_fb_itu.report())
#print(cross_profile_Ericsson_itu.report())
print(cross_profile_facebook_ec.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns  Rows
0  Facebook        6  4598
1       ITU        6  4598

Column Summary
--------------

Number of columns in common: 6
Number of columns in Facebook but not in ITU: 0
Number of columns in ITU but not in Facebook: 0

Row Summary
-----------

Matched on: giga_school_id
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 4,598
Number of rows in Facebook but not in ITU: 0
Number of rows in ITU but not in Facebook: 0

Number of rows with some compared columns unequal: 2,744
Number of rows with all compared columns equal: 1,854

Column Comparison
-----------------

Number of columns compared with some values unequal: 5
Number of columns compared with all values equal: 1
Total number of values which compare unequal: 9,820

Columns with Unequal Values or Types
------------------------------------

          Column Facebook dtype ITU dtype

In [39]:
# merging and selecting the most frequent values across the sources on each row and colunm
# creating a master coverage dataset and merging with the master school location dataset

In [109]:
import statistics as stats
df_c = pd.concat([d_fc,d_ec,d_it])
coverage = df_c.groupby(['giga_school_id'],as_index=False).agg(lambda x:x.value_counts().index[0])
coverage['coverage_type'] = coverage.apply((lambda x: coverage_type(x)),axis=1)
#coverage = df_c.groupby(['giga_school_id'],as_index=False).agg(lambda x: stats.mode(x)[0][0])


In [110]:
coverage.head()

,giga_school_id,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage
0,G_CRI_000001,YES,4G,YES,YES,YES
1,G_CRI_000002,YES,4G,YES,YES,YES
2,G_CRI_000003,YES,4G,YES,YES,YES
3,G_CRI_000004,YES,4G,YES,YES,YES
4,G_CRI_000005,YES,4G,YES,YES,YES


In [113]:
coverage[coverage['coverage']=='NO'].head(10)

,giga_school_id,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage


In [97]:
#coverage.loc[coverage['coverage_type'] == 'None', 'coverage_type'] = d_gs.coverage_type

In [106]:
''''
#Main
import statistics as stats
df_c = pd.concat([d_fc,d_ec,d_it])
coverage = df_c.groupby(['giga_school_id'],as_index=False).agg(lambda x: stats.mode(x)[0][0])
coverage.loc[coverage['coverage'] == 'Y', 'coverage'] = 'YES'
coverage.loc[coverage['coverage'] == 'N', 'coverage'] = 'NO'
coverage.loc[coverage['2G_coverage'] == 'Y', '2G_coverage'] = 'YES'
coverage.loc[coverage['2G_coverage'] == 'N', '2G_coverage'] = 'NO'
coverage.loc[coverage['3G_coverage'] == 'Y', '3G_coverage'] = 'YES'
coverage.loc[coverage['3G_coverage'] == 'N', '3G_coverage'] = 'NO'
coverage.loc[coverage['4G_coverage'] == 'Y', '4G_coverage'] = 'YES'
coverage.loc[coverage['4G_coverage'] == 'N', '4G_coverage'] = 'NO'

coverage.loc[coverage['coverage_type'] == '4', 'coverage_type'] = '4G'
coverage.loc[coverage['coverage_type'] == '3', 'coverage_type'] = '3G'
coverage.loc[coverage['coverage_type'] == '2', 'coverage_type'] = '2G'
coverage.loc[coverage['coverage_type'] == 'N', 'coverage_type'] = 'None'

coverage.loc[coverage['coverage_type'] == 'None', 'coverage_type'] = d_gs.coverage_type

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-106-4b2080777d29>, line 20)

In [112]:
master.head()

,uuid,country,source,school_id,giga_school_id,name,education_level,student_count,connectivity,type_connectivity,connectivity_speed,electricity_availability,lat,lon
0,4311e67f-667b-35b3-b27b-5eff74a4bfad,Costa Rica,government,3945,G_CRI_000001,LICEO NAPOLEON QUESADA SALAZAR,secondary,1164.0,YES,FO,60.0,Yes,9.943509,-84.061523
1,edf0833e-ad77-32b0-8e80-0c29feceb0a1,Costa Rica,government,3946,G_CRI_000002,LICEO MAURO FERNANDEZ ACUNA,secondary,881.0,YES,FO,40.0,Yes,9.957542,-84.080789
2,561d425c-4e7a-37ab-b96f-a8781095cb82,Costa Rica,government,3948,G_CRI_000003,LICEO ROBERTO BRENES MESEN,secondary,874.0,YES,ADSL,10.0,Yes,9.921953,-84.103735
3,73c10e7a-a1d7-37ae-b703-9a1058d1e958,Costa Rica,government,3982,G_CRI_000004,LICEO MONSENOR RUBEN ODIO HERRERA,secondary,1519.0,YES,FO,105.0,Yes,9.901808,-84.067074
4,8a870607-dcf8-3ea4-b356-0ada8e5124bb,Costa Rica,government,3997,G_CRI_000005,LICEO PURISCAL,secondary,930.0,YES,FO,60.0,Yes,9.847161,-84.313211


In [114]:
master_w_coverage = master.merge(coverage, how="left", on=["giga_school_id"])
master_w_coverage.to_csv('./data/CR_master_w_coverage.csv', encoding='utf-8')
master_w_coverage_connectivity = master_w_coverage.copy()
#master_w_coverage_connectivity['connectivity_availability'] = np.where((master_w_coverage_connectivity['connectivity_type'] != 'None'), 'YES', 'NO')
master_w_coverage_connectivity.loc[master_w_coverage_connectivity['coverage'] == 'NO', 'connectivity'] = 'NO'
master_w_coverage_connectivity.loc[master_w_coverage_connectivity['coverage'] == 'NO', 'coverage_type'] = 'None'
master_w_coverage_connectivity.loc[master_w_coverage_connectivity['coverage_type'] == 'None', 'coverage'] = 'NO'
master_w_coverage_connectivity.loc[master_w_coverage_connectivity['coverage_type'] == 'None', 'coverage_type'] = 'no coverage'
master_w_coverage_connectivity.to_csv('./data/CR_master_w_coverage_connectivity.csv', encoding='utf-8')

In [115]:
master_w_coverage.head(2)

,uuid,country,source,school_id,giga_school_id,name,education_level,student_count,connectivity,type_connectivity,connectivity_speed,electricity_availability,lat,lon,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage
0,4311e67f-667b-35b3-b27b-5eff74a4bfad,Costa Rica,government,3945,G_CRI_000001,LICEO NAPOLEON QUESADA SALAZAR,secondary,1164.0,YES,FO,60.0,Yes,9.943509,-84.061523,YES,4G,YES,YES,YES
1,edf0833e-ad77-32b0-8e80-0c29feceb0a1,Costa Rica,government,3946,G_CRI_000002,LICEO MAURO FERNANDEZ ACUNA,secondary,881.0,YES,FO,40.0,Yes,9.957542,-84.080789,YES,4G,YES,YES,YES


In [116]:
master_w_coverage_connectivity[master_w_coverage_connectivity['coverage_type']=='no coverage'].head(10)

,uuid,country,source,school_id,giga_school_id,name,education_level,student_count,connectivity,type_connectivity,connectivity_speed,electricity_availability,lat,lon,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage


In [117]:
master_w_coverage_connectivity.head()

,uuid,country,source,school_id,giga_school_id,name,education_level,student_count,connectivity,type_connectivity,connectivity_speed,electricity_availability,lat,lon,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage
0,4311e67f-667b-35b3-b27b-5eff74a4bfad,Costa Rica,government,3945,G_CRI_000001,LICEO NAPOLEON QUESADA SALAZAR,secondary,1164.0,YES,FO,60.0,Yes,9.943509,-84.061523,YES,4G,YES,YES,YES
1,edf0833e-ad77-32b0-8e80-0c29feceb0a1,Costa Rica,government,3946,G_CRI_000002,LICEO MAURO FERNANDEZ ACUNA,secondary,881.0,YES,FO,40.0,Yes,9.957542,-84.080789,YES,4G,YES,YES,YES
2,561d425c-4e7a-37ab-b96f-a8781095cb82,Costa Rica,government,3948,G_CRI_000003,LICEO ROBERTO BRENES MESEN,secondary,874.0,YES,ADSL,10.0,Yes,9.921953,-84.103735,YES,4G,YES,YES,YES
3,73c10e7a-a1d7-37ae-b703-9a1058d1e958,Costa Rica,government,3982,G_CRI_000004,LICEO MONSENOR RUBEN ODIO HERRERA,secondary,1519.0,YES,FO,105.0,Yes,9.901808,-84.067074,YES,4G,YES,YES,YES
4,8a870607-dcf8-3ea4-b356-0ada8e5124bb,Costa Rica,government,3997,G_CRI_000005,LICEO PURISCAL,secondary,930.0,YES,FO,60.0,Yes,9.847161,-84.313211,YES,4G,YES,YES,YES


In [118]:
master_w_coverage_connectivity['coverage'].value_counts()

YES    4598
Name: coverage, dtype: int64